In [2]:
# Segmentation Functions
import sys
sys.path.insert(0, '/home/ec2-user/SageMaker/Segment2P')

from segment_functions import *
#--------------------------------------#

Using TensorFlow backend.


In [4]:
data_dir = "/home/ec2-user/SageMaker/images/"
model_path_1 = "/home/mestalbet/Segment2P/TrainWeights/mask_rcnn_cell_0100.h5"
model_path_2 = "/home/mestalbet/Segment2P/TrainWeights/mask_rcnn_cell_0198.h5"
model_path_3 = "/home/mestalbet/Segment2P/TrainWeights/mask_rcnn_cell_0199.h5"
model_path_4 = "/home/mestalbet/Segment2P/TrainWeights/mask_rcnn_cell_0200.h5"
model_list = [model_path_1, model_path_2, model_path_3, model_path_4]

In [ ]:
# Define Configuration class
chunksize = 1
class CellInferenceConfig(cellConfig):
        # Set batch size to 1 to run one image at a time
        GPU_COUNT = 1
        IMAGES_PER_GPU = chunksize
        # Don't resize imager for inferencing
        IMAGE_RESIZE_MODE = "pad64"
        # Non-max suppression threshold to filter RPN proposals.
        # You can increase this during training to generate more propsals.
        RPN_NMS_THRESHOLD = 0.7
        # define the folder path to data for prediction
        global data_dir
        all_files = []
        sub_directory = []
        for root, dirs, files in os.walk(data_dir):
            for file in files:
                relativePath = os.path.relpath(root, data_dir)
                if relativePath == ".":
                    relativePath = ""
                all_files.append(
                    (relativePath.count(os.path.sep), relativePath, file))
        all_files.sort(reverse=True)
        for (count, folder), files in groupby(all_files, itemgetter(0, 1)):
            sub_directory.append(folder)

In [ ]:
##### Save Each Model (different checkpoints)#####
for modelpath in model_list: 
    # Load model
    K.clear_session()
    model = modellib.MaskRCNN(
        mode="inference", config=CellInferenceConfig(), model_dir=data_dir)
    # Load weights from H5
    model.load_weights(modelpath, by_name=True)
    sess = K.get_session()

    outputs = [output.name for output in model.keras_model.outputs]
    outs = {str(o):sess.graph.get_tensor_by_name(o) for o in outputs}
    output_names_all = [output.split(':')[0] for output in outputs]
    
    # Save model
    tf.saved_model.simple_save(sess,
                               model_dirpath+"/savedmodel_%s/" % modelpath.split('/')[-1][-7:-3],
                               inputs={'input_image':model.keras_model.inputs[0], 
                                       'input_image_meta':model.keras_model.inputs[1],
                                       'input_anchors':model.keras_model.inputs[2]},
                               outputs=outs)